Import the biggest dataframe with maximum available books/reviews

In [2]:
import pandas as pd
from datetime import datetime
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144
from sklearn import base
from sklearn.pipeline import Pipeline
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from spacy.en import English
nlp = English()
from spacy.en import STOP_WORDS
from sklearn.linear_model import Ridge

In [3]:
books = pd.read_pickle("bigger_romance.pkl")

In [4]:
books.head()

,authors,awards,date,description,isbn,link,num_ratings,num_reviews,rating,review_dates,review_ids,review_texts,reviewer_names,reviewer_ratings,reviewer_scores,reviews_together,title,year
40,[Jennifer Crusie],[All About Romance (AAR) Annual Reader Poll fo...,December 1st 2006,"For Nina Askew, turning forty means freedom--f...",037377138X,http://www.goodreads.com/book/show/33732.Anyon...,17661,928,3.8,"[Mar 11, 2010, Sep 20, 2009, Feb 27, 2009, Dec...","[review_93605589, review_71904939, review_4772...",[[Opening Line: “The last thing Nina Askew nee...,"[Buggy, Shawna, Kathrynn, Ⓐlleskelle - teamSør...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 5, 3, 5, 5, 4, 3, 4, 4, 5, 5, 3, 4, ...",Opening Line: “The last thing Nina Askew neede...,Anyone But You,2006.0
45,[Rachel Gibson],[OKRWA National Readers Choice Award for Singl...,January 1st 1998,Georgeanne Howard leaves her fiancé at the alt...,0380790076,http://www.goodreads.com/book/show/60220.Simpl...,18156,635,3.86,"[Oct 20, 2010, Sep 30, 2009, Apr 04, 2017, Jun...","[review_127198135, review_73023830, review_195...",[[3.5 STARSI must admit I was a bit disappoint...,"[valee, Auntee, Nenia *The Flagrant Liberal* C...","[liked it, it was amazing, it was ok, it was o...","[3, 5, 2, 2, 4, 4, 3, 4, 4, 5, 4, 5, 4, 3, 5, ...",3.5 STARSI must admit I was a bit disappointed...,Simply Irresistible\n \n (Chinoo...,1998.0
59,[Kallypso Masters],[],December 13th 2012,Librarian's Note: This is an alternate cover f...,1481152718,http://www.goodreads.com/book/show/17325954-no...,11078,579,4.31,"[Apr 20, 2013, Nov 28, 2011, Nov 14, 2012, Jun...","[review_595540086, review_240655375, review_45...",[[1 star. DNF at 52 %*Emotional porn at its wo...,"[Baba ♥♥♥ Tyler, Marcus, Archer, Dean, Adrian...","[did not like it, it was amazing, it was ok, r...","[1, 5, 2, 4, 5, 1, 5, 2, 4, 5, 5, 5, 5, 5, 5, ...","[ in the hospital (hide spoiler)] Ka-Thunk ,Ka...",Nobody's Hero\n \n (Rescue Me Sa...,2012.0
61,[Kallypso Masters],[],January 24th 2013,NOBODY'S PERFECT is the continuing story of Sa...,1480096954,http://www.goodreads.com/book/show/17299927-no...,8462,506,4.34,"[Jun 09, 2012, Feb 25, 2012, May 28, 2012, Mar...","[review_345240483, review_283985473, review_33...","[[Nobody's perfect, mi sueño, but you're about...","[~ Becs ~, CaroleDee, Nia, Becca, Amy, Michael...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 2, 2, 5, 2, 5, 4, 5, 1, 5, 3, 5, 4, ...","Nobody's perfect, mi sueño, but you're about a...",Nobody's Perfect\n \n (Rescue Me...,2013.0
66,[Kallypso Masters],[],December 22nd 2013,When Marc d’Alessio first rescued the curvaceo...,NA,http://www.goodreads.com/book/show/17207097-so...,3940,331,4.28,"[Jan 04, 2013, Nov 12, 2013, Jan 02, 2014, Dec...","[review_494970690, review_763151413, review_80...","[[I like this series, but this one was a littl...","[Vivian, Sylvia, Morgan, Sue, Rebekah, Dianne,...","[liked it, did not like it, it was ok, really ...","[3, 1, 2, 4, 2, 4, 2, 4, 2, 5, 5, 5, 4, 5, 4, ...",It's LIVE! It's live! It's finally live! OMG! ...,Somebody's Angel\n \n (Rescue Me...,2013.0


In [5]:
stars = books['rating'].tolist()

In [6]:
stars= np.array(stars).astype(np.float)

In [7]:
stars[:10]

array([ 3.8 ,  3.86,  4.31,  4.34,  4.28,  4.08,  4.17,  3.81,  4.24,  3.96])

In [8]:
num_reviews=books['num_reviews'].tolist()

In [9]:
num_ratings=books['num_ratings'].tolist()

In [10]:
descriptions = books['description'].tolist()

In [11]:
dates=books['date'].tolist()

In [12]:
len(dates)

11018

In [13]:
engagement=pd.DataFrame(
    {'num_reviews': num_reviews,
     'num_ratings': num_ratings,
     'descriptions': descriptions,
     'dates':dates
    })

In [14]:
engagement.head()

,dates,descriptions,num_ratings,num_reviews
0,December 1st 2006,"For Nina Askew, turning forty means freedom--f...",17661,928
1,January 1st 1998,Georgeanne Howard leaves her fiancé at the alt...,18156,635
2,December 13th 2012,Librarian's Note: This is an alternate cover f...,11078,579
3,January 24th 2013,NOBODY'S PERFECT is the continuing story of Sa...,8462,506
4,December 22nd 2013,When Marc d’Alessio first rescued the curvaceo...,3940,331


In [15]:
no_blanks=engagement[engagement["num_ratings"]!='']

In [16]:
engagement.shape

(11018, 4)

In [17]:
no_blanks.shape

(8023, 4)

In [18]:
no_blanks['engagment']=no_blanks['num_ratings'] + no_blanks['num_reviews']

/Users/brechann/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
no_blanks.head()

,dates,descriptions,num_ratings,num_reviews,engagment
0,December 1st 2006,"For Nina Askew, turning forty means freedom--f...",17661,928,18589
1,January 1st 1998,Georgeanne Howard leaves her fiancé at the alt...,18156,635,18791
2,December 13th 2012,Librarian's Note: This is an alternate cover f...,11078,579,11657
3,January 24th 2013,NOBODY'S PERFECT is the continuing story of Sa...,8462,506,8968
4,December 22nd 2013,When Marc d’Alessio first rescued the curvaceo...,3940,331,4271


In [20]:
no_blanks["dts"]=pd.to_datetime(no_blanks["dates"], errors='coerce')

/Users/brechann/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
datetime_object = datetime.strptime('Oct 17 2017', '%b %d %Y')

In [22]:
beginning=datetime.strptime('Jan 1 2007', '%b %d %Y')

In [23]:
datetime_object

datetime.datetime(2017, 10, 17, 0, 0)

In [24]:
datetime_object-beginning

datetime.timedelta(3942)

In [25]:
no_blanks['period']=(datetime_object-no_blanks['dts'])

/Users/brechann/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
no_blanks['days']=(no_blanks['period']).dt.days

/Users/brechann/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
a = np.array(no_blanks['days'].values.tolist())

In [28]:
no_blanks['days_corrected'] = np.where(a > 3942, 3942, a).tolist()

/Users/brechann/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.
/Users/brechann/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
no_blanks['frequency']=no_blanks['engagment']/no_blanks['days_corrected']

/Users/brechann/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
no_blanks['frequency'].std(skipna=True)

66.811461146464552

In [31]:
no_blanks['frequency'].max()

1961.975216365067

In [32]:
no_blanks['frequency'].min()

-2.6666666666666665

In [33]:
books['rating'].max()

'5.0'

In [34]:
books['rating'].min()

'0.0'

In [35]:
final=no_blanks[no_blanks["frequency"]>0]

In [36]:
#final.to_pickle('engagement.pkl')

In [37]:
frequency=final['frequency'].tolist()

In [38]:
frequency= np.array(frequency).astype(np.float)

In [39]:
freq_descriptions=final['descriptions'].tolist()

In [40]:
len(freq_descriptions)

7671

In [41]:
parameters = {
   #'vectorize__min_df': (5,10),
   #'vectorize__max_df': (0.95,0.97,0.99),
   # 'vectorize__stop_words':(STOP_WORDS,None)
   # 'vectorize__max_features':(3000,4000,5000),
    #'ridge__fit_intercept' : (True,False)
    'ridge__alpha':(0.95,1)
}


In [42]:
pipeline = Pipeline([
     # Vectorizer
        ('vectorize', TfidfVectorizer()),
    #regularized linear model--Ridge
        ('ridge',Ridge())
    ])

In [43]:
grid_search = GridSearchCV(pipeline, parameters, verbose=1,n_jobs=2)
grid_search.fit(freq_descriptions, frequency)


Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=2)]: Done   6 out of   6 | elapsed:   10.6s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tru...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'ridge__alpha': (0.95, 1)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=1)

In [44]:
grid_search.best_params_

{'ridge__alpha': 0.95}

In [45]:
grid_search.cv_results_

/Users/brechann/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/brechann/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/brechann/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/brechann/anaconda/li

{'mean_fit_time': array([ 1.87562807,  1.28781827]),
 'mean_score_time': array([ 0.70604467,  0.50038973]),
 'mean_test_score': array([ 0.21860848,  0.21729674]),
 'mean_train_score': array([ 0.74366212,  0.73385146]),
 'param_ridge__alpha': masked_array(data = [0.95 1],
              mask = [False False],
        fill_value = ?),
 'params': [{'ridge__alpha': 0.95}, {'ridge__alpha': 1}],
 'rank_test_score': array([1, 2], dtype=int32),
 'split0_test_score': array([ 0.27950567,  0.27776093]),
 'split0_train_score': array([ 0.74132359,  0.7318081 ]),
 'split1_test_score': array([ 0.13824829,  0.137782  ]),
 'split1_train_score': array([ 0.73422078,  0.7239336 ]),
 'split2_test_score': array([ 0.23807149,  0.23634729]),
 'split2_train_score': array([ 0.755442  ,  0.74581269]),
 'std_fit_time': array([ 0.34366385,  0.04167047]),
 'std_score_time': array([ 0.14984918,  0.01102452]),
 'std_test_score': array([ 0.05928754,  0.05871239]),
 'std_train_score': array([ 0.00881993,  0.00904821])}

In [46]:
grid_search.cv_results_

/Users/brechann/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/brechann/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/brechann/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/brechann/anaconda/li

{'mean_fit_time': array([ 1.87562807,  1.28781827]),
 'mean_score_time': array([ 0.70604467,  0.50038973]),
 'mean_test_score': array([ 0.21860848,  0.21729674]),
 'mean_train_score': array([ 0.74366212,  0.73385146]),
 'param_ridge__alpha': masked_array(data = [0.95 1],
              mask = [False False],
        fill_value = ?),
 'params': [{'ridge__alpha': 0.95}, {'ridge__alpha': 1}],
 'rank_test_score': array([1, 2], dtype=int32),
 'split0_test_score': array([ 0.27950567,  0.27776093]),
 'split0_train_score': array([ 0.74132359,  0.7318081 ]),
 'split1_test_score': array([ 0.13824829,  0.137782  ]),
 'split1_train_score': array([ 0.73422078,  0.7239336 ]),
 'split2_test_score': array([ 0.23807149,  0.23634729]),
 'split2_train_score': array([ 0.755442  ,  0.74581269]),
 'std_fit_time': array([ 0.34366385,  0.04167047]),
 'std_score_time': array([ 0.14984918,  0.01102452]),
 'std_test_score': array([ 0.05928754,  0.05871239]),
 'std_train_score': array([ 0.00881993,  0.00904821])}

In [47]:
grid_search.best_params_

{'ridge__alpha': 0.95}

In [48]:
bag_words_est = Pipeline([
     # Vectorizer
        ('vectorize',  TfidfVectorizer()),
    #regularized linear model--Ridge
        ('ridge',Ridge(alpha=0.95))
])

In [49]:
bag_words_est.fit(freq_descriptions, frequency)

Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tru...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [51]:
frequency

array([  4.71562659,   4.76686961,   6.58959864, ...,   1.09536178,
        14.58037952,   4.08107117])

In [52]:
np.median(frequency)

1.4528629265471371

In [53]:
np.percentile(frequency,88)

14.623355143584153

In [54]:
np.percentile(frequency,87)

13.444026341092155

In [55]:
fake_description='Lisa, a bold aspiring actress, moves to Italy at Christmas to get some inpiration and pasta! She has a pact with her best friend that they will focus on career. But then she meets Rob, a magazine editor from California. Sparks fly! Will Lisa find her forever home in Italy? Will Rob propose? Will they overcome the tribulations and find happiness? Nominated for the predictability award, this book is part of the cliche series.'

In [56]:
example="One more game. That's all that stands between NHL star Asher Westmore and a major career milestone. But then his brother bashes him against the hockey boards, sidelining him for months. Over Christmas holidays, no less. If staying off the ice doesn't drive him crazy, staying with his family will. The only bright spot amidst the mistletoe and twinkle lights: His best friend, Emma, is a physical therapist. And working out with her improves his body and his spirit."

In [68]:
test=[fake_description,example]

In [69]:
test

['Lisa, a bold aspiring actress, moves to Italy at Christmas to get some inpiration and pasta! She has a pact with her best friend that they will focus on career. But then she meets Rob, a magazine editor from California. Sparks fly! Will Lisa find her forever home in Italy? Will Rob propose? Will they overcome the tribulations and find happiness? Nominated for the predictability award, this book is part of the cliche series.',
 "One more game. That's all that stands between NHL star Asher Westmore and a major career milestone. But then his brother bashes him against the hockey boards, sidelining him for months. Over Christmas holidays, no less. If staying off the ice doesn't drive him crazy, staying with his family will. The only bright spot amidst the mistletoe and twinkle lights: His best friend, Emma, is a physical therapist. And working out with her improves his body and his spirit."]

In [70]:
bag_words_est.predict(test)

array([ 14.50987955,  29.58713797])

In [62]:
grid_search2 = GridSearchCV(pipeline, parameters, verbose=1,n_jobs=2)

#print("Performing grid search...")
#print("pipeline:", [name for name, _ in pipeline.steps])
#print("parameters:")
#pprint(parameters)
#t0 = time()
grid_search2.fit(descriptions, stars)
#print("done in %0.3fs" % (time() - t0))
#print()

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=2)]: Done   6 out of   6 | elapsed:   16.7s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tru...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'ridge__alpha': (0.95, 1)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=1)

In [63]:
grid_search2.grid_scores_

/Users/brechann/anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.02307, std: 0.01408, params: {'ridge__alpha': 0.95},
 mean: 0.02573, std: 0.01366, params: {'ridge__alpha': 1}]

In [64]:
grid_search2.cv_results_

/Users/brechann/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/brechann/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/brechann/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/brechann/anaconda/li

{'mean_fit_time': array([ 2.73187065,  2.43873906]),
 'mean_score_time': array([ 0.9845957 ,  0.87780762]),
 'mean_test_score': array([ 0.02306521,  0.0257317 ]),
 'mean_train_score': array([ 0.62569695,  0.61581221]),
 'param_ridge__alpha': masked_array(data = [0.95 1],
              mask = [False False],
        fill_value = ?),
 'params': [{'ridge__alpha': 0.95}, {'ridge__alpha': 1}],
 'rank_test_score': array([2, 1], dtype=int32),
 'split0_test_score': array([ 0.00441084,  0.00777019]),
 'split0_train_score': array([ 0.61292861,  0.60331407]),
 'split1_test_score': array([ 0.03842389,  0.04087446]),
 'split1_train_score': array([ 0.61222782,  0.60265653]),
 'split2_test_score': array([ 0.02636179,  0.02855121]),
 'split2_train_score': array([ 0.65193441,  0.64146602]),
 'std_fit_time': array([ 0.130468  ,  0.07396803]),
 'std_score_time': array([ 0.11734387,  0.04808389]),
 'std_test_score': array([ 0.01408063,  0.01366159]),
 'std_train_score': array([ 0.01855489,  0.01814197])}

In [65]:
bag_of_words_est = Pipeline([
     # Vectorizer
        ('vectorize',  TfidfVectorizer(min_df=0.001,max_df=0.99, stop_words=STOP_WORDS)),
    #regularized linear model--Ridge
        ('ridge',Ridge(alpha=100))
    
    
    # Column selector (remember the ML project?)
    # Vectorizer
    # Frequency filter (if necessary)
    # Regressor
])

In [66]:
bag_of_words_est.fit(descriptions, stars)

Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.99, max_features=None, min_df=0.001,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_id...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [71]:
bag_of_words_est.predict(test)

array([ 4.01761213,  4.01156932])

In [72]:
new_books = pd.read_pickle("modern_romance.pkl")

In [73]:
new_books[:10]

,authors,awards,date,description,isbn,link,page,rating,review_dates,review_ids,review_texts,reviewer_names,reviewer_ratings,reviewer_scores,title
0,[Becky Albertalli],[],April 11th 2017,Seventeen-year-old Molly Peskin-Suso knows all...,0062348701,http://www.goodreads.com/book/show/30653853-th...,"<!DOCTYPE html>\n<html class=""desktop"">\n\n\n<...",4.17,"[May 12, 2017, May 12, 2017, May 12, 2017, May...","[review_1998467435, review_1997758994, review_...",[[3.5 Stars. Just like Simon Vs. The Homo Sapi...,"[Sophia Jones, Kaitie, Ashley Peet, Azee, Tiff...","[liked it, really liked it, it was amazing, it...","[3, 4, 5, 5, 5, 2, 4, 4, 3, 4, 5, 4, 4, 5, 5, ...",The Upside of Unrequited
1,[Jenn Bennett],[],April 4th 2017,The one guy Bailey Rydell can’t stand is actua...,148147877X,http://www.goodreads.com/book/show/30312700-al...,"<!DOCTYPE html>\n<html class=""desktop"">\n\n\n<...",4.07,"[May 12, 2017, May 12, 2017, May 12, 2017, May...","[review_1998055552, review_1907935087, review_...","[[3.5 Stars!], [3.75 STARS], [[ one thing the ...","[Pehel , Laura, Erin, Love, Beck, Melissa, Mar...","[liked it, really liked it, liked it, it was a...","[3, 4, 3, 5, 5, 4, 5, 4, 3, 3, 5, 4, 4, 4, 4, ...","Alex, Approximately"
2,[Ashley Poston],[],April 4th 2017,Anything can happen once upon a con…When geek ...,1594749477,http://www.goodreads.com/book/show/30724132-ge...,"<!DOCTYPE html>\n<html class=""desktop"">\n\n\n<...",4.15,"[May 12, 2017, May 12, 2017, May 12, 2017, May...","[review_1998509105, review_1952345977, review_...","[[4.5 stars! THIS BOOK, it's absolutely adorab...","[Giselle (onwednesdaysweread), Whitney, Alisa,...","[it was amazing, really liked it, really liked...","[5, 4, 4, 3, 5, 4, 5, 3, 5, 4, 5, 5, 5, 5, 4, ...",Geekerella
3,"[Kylie Scott, Kylie Scott]",[],April 11th 2017,When his younger brother loses interest in onl...,1250083222,http://www.goodreads.com/book/show/28220678-twist,"<!DOCTYPE html>\n<html class=""desktop"">\n\n\n<...",3.98,"[May 12, 2017, May 12, 2017, May 12, 2017, May...","[review_1642584644, review_1802394005, review_...","[[just was not as good as the first.], [Lumber...","[Tina, Olivia, Holly Hawk, Caraleigh Tackett, ...","[liked it, it was amazing, it was amazing, it ...","[3, 5, 5, 5, 3, 4, 3, 4, 5, 3, 4, 2, 4, 5, 4, ...",Twist\n \n (Dive Bar #2)
4,[Melissa de la Cruz],[],April 11th 2017,Their romance shaped a nation. The rest was hi...,1524739626,http://www.goodreads.com/book/show/32860355-al...,"<!DOCTYPE html>\n<html class=""desktop"">\n\n\n<...",3.72,"[May 12, 2017, May 12, 2017, May 12, 2017, May...","[review_1958296739, review_1932197856, review_...","[[Review to come later if I feel like it.], [T...","[Faith, Andrea , Katie, Renata, Amy, Michelle ...","[liked it, it was ok, really liked it, it was ...","[3, 2, 4, 2, 4, 2, 5, 5, 3, 3, 5, 1, 3, 2, 5, ...",Alex and Eliza
5,[Claudia Gray],[],April 4th 2017,She’s a soldier.Noemi Vidal is seventeen years...,0316394033,http://www.goodreads.com/book/show/31423196-de...,"<!DOCTYPE html>\n<html class=""desktop"">\n\n\n<...",4.12,"[May 12, 2017, May 12, 2017, May 12, 2017, May...","[review_1998352947, review_1956220352, review_...",[[2.5 starsI'm beginning to think Gray and I a...,"[Anna, Vidhi, DeA, Khurram, Reading Teen, Brid...","[liked it, it was amazing, it was amazing, rea...","[3, 5, 5, 4, 4, 4, 4, 4, 5, 3, 4, 3, 5, 3, 4, ...",Defy the Stars\n \n (Constellati...
6,[Julie James],[],April 18th 2017,Two undercover FBI agents can hide who they ar...,0425273776,http://www.goodreads.com/book/show/27246123-th...,"<!DOCTYPE html>\n<html class=""desktop"">\n\n<he...",4.0,"[May 12, 2017, May 12, 2017, May 11, 2017, May...","[review_1991333055, review_1579824681, review_...",[[Very disappointed. I have every book she's w...,"[Mary Lemon, Tenshi, Patricia Camaioni, Holly ...","[liked it, liked it, liked it, it was amazing,...","[3, 3, 3, 5, 4, 5, 4, 5, 3, 1, 5, 2, 2, 2, 4, ...",The Thing About Love
7,[Lexa Hillyer],[],April 11th 2017,A kingdom burns. A prin

In [74]:
newDescriptions = new_books['description'].tolist()

In [75]:
test=newDescriptions[:10]

In [76]:
test

['Seventeen-year-old Molly Peskin-Suso knows all about unrequited love—she’s lived through it twenty-six times. She crushes hard and crushes often, but always in secret. Because no matter how many times her twin sister, Cassie, tells her to woman up, Molly can’t stomach the idea of rejection. So she’s careful. Fat girls always have to be careful.Then a cute new girl enters Cassie’s orbit, and for the first time ever, Molly’s cynical twin is a lovesick mess. Meanwhile, Molly’s totally not dying of loneliness—except for the part where she is. Luckily, Cassie’s new girlfriend comes with a cute hipster-boy sidekick. Will is funny and flirtatious and just might be perfect crush material. Maybe more than crush material. And if Molly can win him over, she’ll get her first kiss and she’ll get her twin back. There’s only one problem: Molly’s coworker Reid. He’s an awkward Tolkien superfan with a season pass to the Ren Faire, and there’s absolutely no way Molly could fall for him. Right?',
 'The

In [78]:
bag_of_words_est.predict(test)

array([ 4.00100816,  4.00786013,  4.00436486,  4.02389606,  4.01071221,
        4.05261576,  4.01938291,  4.02730906,  4.02162133,  4.04691185])